# Lesson 04:  Парсинг HTML. XPath

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
*название источника,
*наименование новости,
*ссылку на новость,
*дата публикации

2)Сложить все новости в БД

In [55]:
from lxml import html
import pandas as pd
import requests
from pprint import pprint
from datetime import date

In [56]:
mail_link = 'https://mail.ru'
lenta_link = 'https://lenta.ru/'
header = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 5_0 like Mac OS X) '
                        'AppleWebKit/534.46 (KHTML, like Gecko) Version/5.1 '
                        'Mobile/9A334 Safari/7534.48.3'}

In [57]:
def request_to_mail(mail_link, header):
    """Function to collect the news data from mail.ru"""
    link = mail_link
    today = date.today()
    response = requests.get(mail_link, headers=header)
    root = html.fromstring(response.text)
    news_blocks = root.xpath("//div[@id='news-1']/a[@class='list__item']")
    news_data =[]

    for news_block in news_blocks:
        news_source = link
        news_title = news_block.xpath(".//span[@class='list__item__title']/text()")[0].replace('\xa0', ' ')
        news_link = news_block.xpath("./@href")[0]
        news_date = today.strftime("%d/%m/%Y")
        
    
        
        news_data.append({
            'news_source': news_source,
            'news_title': news_title,
            'news_link': news_link,
            'news_date': news_date
        })

    pprint(news_data)
    mail_ru_data = pd.DataFrame(news_data)
    mail_ru_data.to_csv('mail_ru_data.csv', encoding='utf-8', index=False)

In [58]:
request_to_mail(main_link, header)

[{'news_date': '23/02/2020',
  'news_link': 'https://r.mail.ru/n320977260?&test_id=63&rnd=12156988',
  'news_source': 'https://mail.ru',
  'news_title': 'Сапарбаев: Руководители школ начали чувствовать себя '
                'князьками'},
 {'news_date': '23/02/2020',
  'news_link': 'https://r.mail.ru/n320990012?&test_id=63&rnd=12156988',
  'news_source': 'https://mail.ru',
  'news_title': 'Туман накроет почти весь Казахстан'},
 {'news_date': '23/02/2020',
  'news_link': 'https://r.mail.ru/n320983288?&test_id=63&rnd=12156988',
  'news_source': 'https://mail.ru',
  'news_title': 'Адвокат осужденных за убийство егеря: Там не было убийства'},
 {'news_date': '23/02/2020',
  'news_link': 'https://r.mail.ru/n320969526?&test_id=63&rnd=12156988',
  'news_source': 'https://mail.ru',
  'news_title': 'Одну из школ Петропавловска закрыли на карантин'},
 {'news_date': '23/02/2020',
  'news_link': 'https://r.mail.ru/n320972734?&test_id=63&rnd=12156988',
  'news_source': 'https://mail.ru',
  'news_tit

In [59]:
def request_to_lenta(lenta_link, header):
    """Function to collect the news data from lenta.ru"""
    link = lenta_link
    today = date.today()
    response = requests.get(lenta_link, headers=header)
    root = html.fromstring(response.text)
    news_blocks = root.xpath("//ul[@class='main-header__topics-list']//li/a")
    news_data =[]

    for news_block in news_blocks:
        news_source = link
        news_title = news_block.xpath("./div[@class='card-mini__text-block']/div[@class='card-mini__title']/text()")[0]
        news_link = 'https://lenta.ru/' + str(news_block.xpath("./@href")[0])
        news_date = today.strftime("%d/%m/%Y")

        news_data.append({
            'news_source': news_source,
            'news_title': news_title,
            'news_link': news_link,
            'news_date': news_date
        })
        
    pprint(news_data)
    
    lenta_ru_data = pd.DataFrame(news_data)
    lenta_ru_data.to_csv('lenta_ru_data.csv', encoding='utf-8', index=False)

In [60]:
request_to_lenta(lenta_link, header)

[{'news_date': '23/02/2020',
  'news_link': 'https://lenta.ru//news/2020/02/23/evertonars/',
  'news_source': 'https://lenta.ru/',
  'news_title': '«Арсенал» одержал волевую победу над «Эвертоном» в матче АПЛ'},
 {'news_date': '23/02/2020',
  'news_link': 'https://lenta.ru//news/2020/02/23/tamozh/',
  'news_source': 'https://lenta.ru/',
  'news_title': 'Замглавы таможенного поста в Шереметьево заподозрили в '
                'получении взятки'},
 {'news_date': '23/02/2020',
  'news_link': 'https://lenta.ru//news/2020/02/23/rossiya/',
  'news_source': 'https://lenta.ru/',
  'news_title': 'Россиянин сломал девушке ребра за невкусный ужин'},
 {'news_date': '23/02/2020',
  'news_link': 'https://lenta.ru//news/2020/02/23/reaction/',
  'news_source': 'https://lenta.ru/',
  'news_title': 'Тренер Логинова отреагировал на информацию о возбуждении '
                'уголовного дела'},
 {'news_date': '23/02/2020',
  'news_link': 'https://lenta.ru//news/2020/02/23/shepelev/',
  'news_source': 'htt

In [61]:
from pymongo import MongoClient

In [68]:
# DB connectivity
client = MongoClient('localhost', 27017)
db = client['news_collection_db']
mail_ru_collection = db.mail_ru_db
lenta_ru_collection = db.lenta_ru_db

In [73]:
def csv_to_json(filename):
    """Function to parse csv to dictionary"""
    data = pd.read_csv(filename, encoding='utf-8')
    return data.to_dict('records')

In [74]:
# To insert mail.ru and lenta.ru news collections into MongoDB
mail_ru_collection.insert_many(csv_to_json('mail_ru_data.csv'))

In [75]:
# To insert lenta.ru news collections into MongoDB
lenta_ru_collection.insert_many(csv_to_json('lenta_ru_data.csv'))